# Продажа квартир в Санкт-Петербурге — анализ рынка недвижимости

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

# 1. Общая информация о файле с данными

- Загрузите данные из файла в датафрейм.
- Изучите общую информацию о полученном датафрейме.
- Постройте общую гистограмму для всех столбцов таблицы.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

data = pd.read_csv('/datasets/real_estate_data.csv',sep='\t')

#выведем на экран первые строки:
display(data.head())

#выясним сколько значений в каждом столбце и формат данных:
data.info() 
   
#построим общую гистограмму для всех столбцов таблицы
data.hist(figsize=(15, 20));


Файл содержит данные по 23699 объектам недвижимости, каждый  объект описывается 22 признаками (17 количественных, 4 категориальных и 1 временной). У некоторых признаков есть пропущенные значения, которые необходимо обработать.

# 2. Предобработка данных

1.	Найдите и изучите пропущенные значения в столбцах:
- Определите, в каких столбцах есть пропуски.
- Заполните пропущенные значения там, где это возможно. Например, если продавец не указал число балконов, то, скорее всего, в его квартире их нет. Такие пропуски правильно заменить на 0. Если логичную замену предложить невозможно, то оставьте эти значения пустыми. Пропуски — тоже важный сигнал, который нужно учитывать.
- В ячейке с типом markdown укажите причины, которые могли привести к пропускам в данных.
2.	Рассмотрите типы данных в каждом столбце:
- Найдите столбцы, в которых нужно изменить тип данных.
- Преобразуйте тип данных в выбранных столбцах.
- В ячейке с типом markdown поясните, почему нужно изменить тип данных.
3.	Изучите уникальные значения в столбце с названиями и устраните неявные дубликаты. Например, «поселок Рябово» и «поселок городского типа Рябово», «поселок Тельмана» и «посёлок Тельмана» — это обозначения одних и тех же населённых пунктов. Вы можете заменить названия в существующем столбце или создать новый с названиями без дубликатов.
4.	Найдите и устраните редкие и выбивающиеся значения. Например, в столбце ceiling_height может быть указана высота потолков 25 м и 32 м. Логично предположить, что на самом деле это вещественные значения: 2.5 м и 3.2 м. Попробуйте обработать аномалии в этом и других столбцах.
- Если природа аномалии понятна и данные действительно искажены, то восстановите корректное значение.
- В противном случае удалите редкие и выбивающиеся значения.
- В ячейке с типом markdown опишите, какие особенности в данных вы обнаружили.



In [2]:
#определим, в каких столбцах и в каком количестве есть пропуски:
data.isnull().sum()

На основании полученных данных, можно сделать следующие выводы:

3. first_day_exposition - object - должен быть тип datetime
5. ceiling_height - есть нулевые значения
6. floors_total - должен быть тип int, т.к. количество этажей должно быть целым значением + есть нулевые значения
9. is_apartment - должен быть тип bool + есть нулевые значения
7. living_area - есть нулевые значения
12. kitchen_area - есть нулевые значения
13. balcony - есть нулевые значения
14. locality_name - есть нулевые значения
15. airports_nearest - есть нулевые значения
16. cityCenters_nearest - есть нулевые значения
17. parks_around3000 - тип должен быть int + есть нулевые значения
18. parks_nearest - есть нулевые значения
19. ponds_around3000 - тип должен быть int + есть нулевые значения
20. ponds_nearest - есть нулевые значения
21. days_exposition - тип должен быть int + есть нулевые значения

In [3]:
# check
# пропущенные значения бары

def pass_value_barh(df):
    try:
        (
            (df.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'space'})
            .query('space > 0')
            .sort_values(by = 'space', ascending = True)
            .plot(kind= 'barh', figsize=(19,6), rot = -5, legend = False, fontsize = 16)
            .set_title('Пример' + "\n", fontsize = 22, color = 'SteelBlue')    
        );    
    except:
        print('пропусков не осталось :) ')

In [4]:
pass_value_barh(data)

In [5]:
#заменим пропуски там, где замена не приведет к искажению статистических свойств выборки и к ухудшению результатов:

#переведем столбец с датой в формат даты без времени, т.к. время отсутствует
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%d')

#заменим пропуски в balcony на 0 и изменим тип данных на int
data['balcony'].value_counts()
data['balcony'] = data['balcony'].fillna(0)
data['balcony'] = data['balcony'].astype('int')

#заменим пропуски в floors_total на 0 и изменим тип данных на int
data['floors_total']=data['floors_total'].fillna(0)
data['floors_total']=data['floors_total'].astype('int')


# в параметре ceiling_height: есть большие значения, медиана от среднего отличается не сильно 


data['ceiling_height'].sort_values().plot(y = 'ceiling_height', kind = 'hist', bins = 30, range=(2,5))
data['ceiling_height'].value_counts()
data['ceiling_height'].describe()
data[data['ceiling_height'] > 4].sort_values('ceiling_height').tail(20)

# в столбце is_apartment все пропущенные значения можно заменить на False:
data['is_apartment'] = data['is_apartment'].fillna(False)

#изменим цену на тип int для удобства обработки этих данных:
data['last_price'] = data['last_price'].astype('int')

#остальные данные с пропущенными значениями оставим как есть
data['total_images'].value_counts()
data['ceiling_height'].value_counts()
data['floor'].value_counts()
data['is_apartment'].value_counts()
data['total_area'].value_counts()
data['rooms'].value_counts()
data['airports_nearest'].value_counts()
data['cityCenters_nearest'].value_counts()
data['parks_around3000'].value_counts()
data['parks_nearest'].value_counts()
data['ponds_around3000'].value_counts()
data['living_area'].value_counts()
data['locality_name'].value_counts()


In [6]:
# выведем на экран все уникальные значения 'locality_name'
print(data['locality_name'].unique())

#устраним неявные дубликаты в названиях населённых пунктов:
data['locality_name'] = data['locality_name'].str.replace('поселок городского типа','городской поселок')
data['locality_name'] = data['locality_name'].str.replace('посёлок','поселок')
data['locality_name'] = data['locality_name'].str.replace('садоводческое некоммерческое товарищество','садовое товарищество ')

In [7]:
# check
data['locality_name'].unique()

In [8]:
# check
data.info()

In [9]:
# выведем на экран все наименования столбцов
print(data.columns)

In [10]:
# функция для подсчёта границ с учетом 1.5 размаха

def quartile_range(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    dfq = data.loc[(data[column] < q3 + 1.5*iqr) & (data[column] > q1 - 1.5*iqr), column]
    return dfq

# столбцы с выбросами

list_blowout= ['last_price', 'total_area', 'kitchen_area', 'living_area', 'days_exposition', 
              'ceiling_height', 'rooms','floors_total']
for col in list_blowout:
    #data[col] = quartile_range(data, col)
    
    
# возвращаем в целочисленный тип число комнат

   data['rooms'] = data['rooms'].fillna(0).astype(int)


In [11]:
data.describe()

In [12]:
# check

# Показатели о кол-ве объявлений в датасете, минимальных и максимальных значениях 
# в выбранных параметрах о продаже квартир
# сырые данные

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area', 'floor',
       'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

In [13]:
# check

# Значения параметров объектов недвижимости на разных квантилях

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  
        'kitchen_area', 'floor',   'floors_total']]
    .quantile([0.01, .5, .90, .976])  
    .style.format("{:,.2f}")
)


Boxplot обобщает данные выборки с использованием 25-го, 50-го и 75-го процентилей.
Можно  получить представление (квартили, медиану и выбросы) в наборе данных взглянув на его диаграмму.

In [14]:
# Box Plot
data.boxplot(['living_area'])

In [15]:
data.boxplot(['kitchen_area'])

In [16]:
data.boxplot(['rooms'])

In [17]:
data.boxplot(['last_price'])

In [18]:
# check
data.info()

In [19]:
#Вявляем и исключаем аномальные значения:

data = data.query ('living_area.isna() | living_area >= 12', engine='python')
print ('после I удаления', data.shape[0])

data = data.query ('kitchen_area.isna() | kitchen_area >= 5',engine='python')
print ('после II удаления', data.shape[0])


data = data.query ('rooms.isna() | rooms >= 1',engine='python')
print ('после III удаления', data.shape[0])


data = data.query ('last_price.isna() | last_price >= 1000000',engine='python')
print ('после IV удаления', data.shape[0])


data = data.query ('days_exposition.isna() | days_exposition >= 10',engine='python')
print ('после V удаления', data.shape[0])


data = data.query ('ceiling_height.isna() | ceiling_height >= 2.5',engine='python')
print ('после VI удаления', data.shape[0])


data = data.query('floors_total.isna() | floors_total <= 33',engine='python')
print ('после VII удаления', data.shape[0])



In [20]:
(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area', 'floor',
       'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

# 3. Добавлены новые столбцы с параметрами


-цена одного квадратного метра.

-день недели публикации объявления (0 — понедельник, 1 — вторник и так далее).

-месяц публикации объявления.

-год публикации объявления.

-тип этажа квартиры (значения — «первый», «последний», «другой»).

-расстояние до центра города в километрах (переведите из м в км и округлите до целых значений).


In [21]:
#посчитаем цену одного квадратного метра и добавим соответствующий столбец
data['price_per_square_meter'] = data['last_price']/data['total_area']

In [22]:
#создадим столбец дня недели
data['weekday_exposition'] = data['first_day_exposition'].dt.weekday

#создадим столбец месяца публикации
data['month_exposition'] = data['first_day_exposition'].dt.month

#создадим столбец года публикации
data['year_exposition'] = data['first_day_exposition'].dt.year

In [23]:
# напишем функцию категоризации по этажам:

def floor_category(row):
    floors_total = row['floors_total']
    floor = row['floor']
    if floor == 1:
        return 'первый'
    elif floor == floors_total:
        return 'последний'
    elif 1 < floor < floors_total:
        return 'другой'
    
#категоризуем этажи с помощью функции  

data['floor_category'] = data.apply(floor_category, axis = 1)

In [24]:

# посчитаем и добавим в таблицу: расстояние в км до центра города
data['centers_km'] = data['cityCenters_nearest']/1000
data['centers_km'] = data['centers_km'].fillna(value=0).astype(int)

In [25]:
# выведем на экран все имеющиеся наименования столбцов
print(data.columns)

# 4. Исследовательский анализ данных:


1.	Изучите следующие параметры объектов:
- общая площадь;
- жилая площадь;
- площадь кухни;
- цена объекта;
- количество комнат;
- высота потолков;
- этаж квартиры;
- тип этажа квартиры («первый», «последний», «другой»);
- общее количество этажей в доме;
- расстояние до центра города в метрах;
- расстояние до ближайшего аэропорта;
- расстояние до ближайшего парка;
- день и месяц публикации объявления.
Постройте отдельные гистограммы для каждого из этих параметров. Опишите все ваши наблюдения по параметрам в ячейке с типом markdown.
2.	Изучите, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней было размещено каждое объявление.
- Постройте гистограмму.
- Посчитайте среднее и медиану.
- В ячейке типа markdown опишите, сколько времени обычно занимает продажа. Какие продажи можно считать быстрыми, а какие — необычно долгими?
3.	Какие факторы больше всего влияют на общую (полную) стоимость объекта?
Изучите, зависит ли цена от:
- общей площади;
- жилой площади;
- площади кухни;
- количества комнат;
- этажа, на котором расположена квартира (первый, последний, другой);
- даты размещения (день недели, месяц, год).
Постройте графики, которые покажут зависимость цены от указанных выше параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы.
4.	Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра. Эти данные можно найти по имени в столбце locality_name.
5.	Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца locality_name и вычислите среднюю цену каждого километра. Опишите, как стоимость объектов зависит от расстояния до центра города.


In [26]:
#изучим общую площадь
data.plot(y ='total_area', kind = 'hist', bins = 100, grid=True, figsize = (7,3), range = (10,120))
plt.xlabel('total_area')
plt.ylabel('quantity')
plt.suptitle('Общая площадь')
data['total_area'].describe()

Распределение данных  по гистограмме нормальное, но есть выбросы.Самые распространенные квартиры в объявлениях площадью от 54 кв.м.

In [27]:
#изучим жилую площадь
data.plot(y = 'living_area', kind = 'hist', bins = 100, grid=True, figsize = (7,3), range = (2,80))
plt.xlabel('living_area')
plt.ylabel('quantity')
plt.suptitle ('Жилая площадь')
data['living_area'].describe()

Распределение данных по гистограмме нормальное, но есть выбросы.
Самые распространенные квартиры в объявлениях с жилой площадью от 31 кв.м.

In [28]:
#изучим площадь кухни

data.plot(y = 'kitchen_area', kind = 'hist', bins = 40, grid=True, range = (1,20), figsize = (8,3))
plt.xlabel('kitchen_area')
plt.ylabel('quantity')
plt.suptitle ('Цена')
data['kitchen_area'].describe()

Самые распространенные квартиры в объявлениях с площадью кухни от 9 кв.м.

In [29]:
#работаем со стоимостью
data.plot(y = 'last_price', kind = 'hist', bins = 100, grid=True, range = (1,12000000), figsize = (8,3))
plt.xlabel('last_price')
plt.ylabel('quantity')
plt.suptitle ('Цена')
data['last_price'].describe()

Распределение данных  по гистограмме нормальное, но есть выбросы.
Самые распространенные квартиры в объявлениях с ценой от 4,8 млн. рублей

In [30]:
#изучим количество комнат
data.plot(y = 'rooms', kind = 'hist', bins = 30, grid=True, range = (0,5),figsize = (7,3))
plt.xlabel('rooms')
plt.ylabel('quantity')
plt.suptitle ('Количество комнат')
data['rooms'].describe()

Распределение данных показывает, что самые распространенные квартиры 2 и 1 комнатные. Максимальное количество комнат- 5.

In [31]:
#изучим высоту потолков
data.plot(y = 'ceiling_height', kind = 'hist', bins = 40, range = (2,4), grid=True, figsize = (7,3))
plt.xlabel('ceiling_height')
plt.ylabel('quantity')
plt.suptitle ('Высота потолков')
data['ceiling_height'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры в объявлениях с высотой потолков от 2 метров,максимальное значение 3,21.

In [32]:
# изучим этаж квартиры
data.plot(y = 'floor', kind = 'hist', bins = 35, grid=True, range =(1,35),figsize = (7,3))
plt.xlabel('floor')
plt.ylabel('quantity')
plt.suptitle ('Этаж квартиры')
data['floor'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры в объявлениях этажностью 4/5 ,максимальное значение 33.

In [33]:
# изучим тип этажа объекта («первый», «последний», «другой»)
data.floor_category.value_counts().plot.bar()
plt.xlabel('floor_category')
plt.ylabel('quantity')
plt.suptitle ('Тип этажа')
data['floor_category'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры в объявлениях  в категории "другие", не 1 и не 2-ой этажи.

In [34]:
#общее количество этажей в доме
data.plot(y = 'floors_total', kind = 'hist', bins = 40, grid=True,range = (1,60),figsize = (7,3))
plt.xlabel('floors_total')
plt.ylabel('quantity')
plt.suptitle ('Количество этажей в доме')
data['floors_total'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры в объявлениях расположены на  10 эт.

In [35]:
#расстояние до центра города в метрах
data.plot(y ='cityCenters_nearest', kind = 'hist', bins = 40, range = (100,66000), grid=True, figsize = (7,3))
plt.xlabel('cityCenters_nearest')
plt.ylabel('quantity')
plt.suptitle ('Расстояние до центра города(метров)')
data['cityCenters_nearest'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры с расстоянием до центра города от 14000 (от 14 км.).

In [36]:
#расстояние до ближайшего аэропорта
data.plot(y = 'airports_nearest', kind = 'hist', bins = 40, range = (0,85000), grid=True, figsize = (7,3))
plt.xlabel('airports_nearest')
plt.ylabel('quantity')
plt.suptitle('Расстояние до ближайшего аэропорта')
data['airports_nearest'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры с расстоянием до аэропорта от 28 тыс. м.

In [37]:
#расстояние до ближайшего парка
data.plot(y = 'parks_nearest', kind = 'hist', bins = 40, range = (1,4000), grid=True, figsize = (7,3))
plt.xlabel('parks_nearest')
plt.ylabel('quantity')
plt.suptitle('Расстояние до ближайшего парка')
data['parks_nearest'].describe()

Распределение данных  по гистограмме нормальное.
Самые распространенные квартиры с расстоянием до парка от 490 метров.

In [38]:
#день публикации объявления

data.plot(y ='weekday_exposition' , kind = 'hist', bins = 40, grid=True, range = (0,6), figsize = (7,3))
plt.xlabel('weekday_exposition')
plt.ylabel('quantity')
plt.suptitle('День публикации объявления')

data['weekday_exposition'].describe()

Распределение данных  по гистограмме нормальное.
Больше всего объявлений размещены в чт/вт, меньше всего сб/вс.

In [39]:
#месяц публикации объявления
data.plot(y = 'month_exposition', kind = 'hist', bins = 60, grid=True, range = (1,12), figsize = (7,3))
plt.xlabel('month_exposition')
plt.ylabel('quantity')
plt.suptitle('Месяц публикации объявления')
data['month_exposition'].describe()

Распределение данных  по гистограмме нормальное.
Больше всего объявлений размещены в феврале/марте, меньше всего в мае и январе, что обусловлено спросом и календарем выходных дней.

Средняя квартира, выставленная на продажу, имеет следующие характеристики: это 2-комнатная квартира с общей площадью от 54 кв. м с потолками от 2 м, проданная по цене от 9 млн рублей.

In [40]:
# Время продажи квартиры (столбец days_exposition)
data.plot(y = 'days_exposition', kind = 'hist', bins = 60, grid=True, range = (0,450), figsize = (7,3))

data['days_exposition'].describe()

# медиана = 128
# среднее = 121

In [41]:
data.describe()

Чаще всего объект продают за 121 день.За 41 день- это быстро, за 183 дня- долго.

In [42]:
pd.pivot_table(data,
               index=['total_area','living_area','kitchen_area','rooms',
                      'floor_category','weekday_exposition','month_exposition', 'year_exposition'],
               values=['last_price'])

#построим таблицу корреляции для "общей картины"

In [43]:
data.corr()

In [44]:
#определим какие факторы больше всего влияют на общую (полную) стоимость объекта (last_price)?
#построим графики, которые покажут зависимость цены от указанных ниже параметров.
#зависимость цены от площади

(
    data.pivot_table(index='total_area', values='last_price')
    .plot(grid=True, style='o', figsize=(7, 5))
)
plt.show()


In [45]:
print('Корреляция цены и площади объекта:')

print(data['total_area'].corr(data['last_price']))


Чем больше площадь объекта, тем выше стоимость. 

In [46]:
# зависимость цены от жилой площади объекта(living_area)
(
    data.pivot_table(index='living_area', values='last_price')
    .plot(grid=True, style='o', figsize=(7, 5))
)
plt.show()

print('Корреляция цены и жилой площади объекта:')

print(data['living_area'].corr(data['last_price']))

Чем больше жилая площадь, тем выше стоимость. Корреляция этих параметров высокая.

In [47]:
#зависимость цены от жилой площади кухни объекта(kitchen_area)

(
    data.pivot_table(index='kitchen_area', values='last_price')
    .plot(grid=True, style='o', figsize=(7, 5))
)
plt.show()
print('Корреляция цены и площади кухни:')

print(data['kitchen_area'].corr(data['last_price']))

Чем больше площадь кухни, тем выше стоимость объекта. 

In [48]:
# зависимость цены  от количества комнат объекта(kitchen_area)

(
    data.pivot_table(index='rooms', values='last_price')
    .plot(grid=True, style='o-', figsize=(7, 5))
)
plt.show()

print('Корреляция цены и количества комнат объекта:')
print(data['rooms'].corr(data['last_price']))


Число комнат влияет на цену (больше комнат - выше цена).

In [49]:
# зависимость цены от типа этажа, на котором расположена квартира (floor_category).
# корреляция определяется только между числовыми переменными.
# можно при необходимости закодировать категорию как число и использовать корреляционную функцию Пирсона df для возврата корреляции.
(
    data.pivot_table(index='floor_category', values='last_price', aggfunc='median')
    .plot(grid=True, style='o-', figsize=(5, 5))
)
plt.show()

Первый этаж непосредственно влияет на стоимость объекта - в среднем квартиры на 1-м этаже значительно дешевле, немного дороже последний этаж. Наибольшая стоимость у объектов, расположенных не на 1-м и не на последнем этаже.

In [50]:
#зависимость цены от дня экспозиции
(
    data.pivot_table(index='weekday_exposition', values='last_price')
    .plot(grid=True, style='o-', figsize=(7, 5))
)
plt.show()

In [51]:
print('Корреляция цены и дня экспозиции:')

print(data['weekday_exposition'].corr(data['last_price']))

Зависимость цены от дней экспозиции: самая высокая цена у объектов с днем экспозиции- вторник, у
размещенных в понедельник и пятницу примерно одинаковая цена.Наименьшая цена приходится на выходные дни, а именно суббота.
    

In [52]:
#зависимость цены от месяца экспозиции объекта
(
    data.pivot_table(index='month_exposition', values='last_price')
    .plot(grid=True, style='o-', figsize=(7, 5))
)
plt.show()

In [53]:
print('Корреляция цены и месяца экспозиции:')

print(data['month_exposition'].corr(data['last_price']))

Судя по графику(за  12 месяцев) есть данные зависимости цены от месяца размещения объявления.
В апреле и сентябре зависимость увеличивается. Это можно объяснить и сезонным спросом, который
растет в сентябре с началом делового сезона, когда переезжают в Петербург работать или учиться.
В январе,марте,октябре происходит снижение зависимости.

In [54]:
#зависимость цены от года экспозиции
(
    data.pivot_table(index='year_exposition', values='last_price')
    .plot(grid=True, style='o-', figsize=(7, 5))
)
plt.show()

In [55]:
print('Корреляция цены и года экспозиции:')
print(data['month_exposition'].corr(data['last_price']))

Общие выводы по зависимости от дня, месяца и года.
День недели: дороже всего квартиры, опубликованные во вторник, дешевле всего - в субботу. Месяц: дороже всего объекты, продающиеся с апреле, сентябре, дешевле всего в июне. Год: квартиры очень сильно подешевели в 2016-2018 годах, что, вызвано некоторым кризисом на рынке недвижимости. С 2018 года цены снова стали повышаться в связи с выходом из кризиса.

In [56]:
#сравниваем среднюю стоимость квадратного метра в топ-10 городов по числу объявлений

city_ten = data['locality_name'].value_counts().head(10)
city_price_per_square_meter = data.query('locality_name in (@city_ten.index)').pivot_table(values='price_per_square_meter',
                                                                                           index='locality_name')
city_price_per_square_meter = city_price_per_square_meter.astype(int)
city_price_per_square_meter.sort_values('price_per_square_meter', ascending=False)

Дороже всего квадратный метр в Санкт-Петербурге - 105684
Дешевле всего квадратный метр в Выборге - 58225

In [57]:
#выбираем Санкт-Петербург, смотрим на график, где указана средняя стоимость для каждого км
(
    data.query('locality_name == "Санкт-Петербург"')
    .pivot_table(index='centers_km', values='last_price')
    .plot(grid=True, style='o-', xlim=(0,15), figsize=(10, 5))
)
plt.show()

Центральная зона - от 8 км и меньше. Дальше 8 км средняя цена на объекты снижается.

# 5. Общий вывод

Мы обработали массив данных, в том числе пропущенные значения (заменили пропуски там, где замена не привела к ухудшению результатов) изменили типы данных на необходимые, устраненили неявные дубликаты в названиях населённых пунктов, устранили редкие и выбивающиеся значения во всех столбцах, посчитали и добавили в таблицу слудующие данные: цена одного квадратного метра, день,  месяц, год публикации объявления,тип этажа квартиры,расстояние в км до центра города. Заменили пропуски там, где замена не привела к ухудшению результатов.

Изучили и построили гистограммы для следующих параметров: общая площадь; жилая площадь;площадь кухни;цена объекта;количество комнат;высота потолков;этаж квартиры;тип этажа квартиры («первый», «последний», «другой»);общее количество этажей в доме;расстояние до центра города в метрах;расстояние до ближайшего аэропорта;расстояние до ближайшего парка;день и месяц публикации объявления.

Выявили что чаще всего объект продают за 85 дней. За 41 день- это быстро, за 183 дня- долго. 
Есть квартиры, проданные всего за несколько дней после публикации.
Также есть варианты, которые продавались несколько месяцев(долго).

Выявили факторы,влияющие на стоимость и построили графики корреляции.
На стоимость прежде всего влияет площадь квартиры. Первый этаж значительно дешевле остальных вариантов. 
Также стоимость квартиры на последнем этаже ниже, чем на остальных, кроме первого.
  
Стоимость квадратного метра в топ-10 населённых пунктов
Дороже всего квадратный метр в Санкт-Петербурге - 105684
Дешевле всего квадратный метр в Выборге - 58225.


Опрелили зависимость стоимости объектов в Санкт-Петербурге  от расстояния до центра города
Центральная зона - от 8 км и меньше. Дальше 8 км цена на объекты в Санкт-Петербурге снижается.

In [58]:
# check
import seaborn as sns

In [59]:
# check
sns.pairplot(data[['last_price', 'total_area', 'rooms', 'cityCenters_nearest']])
plt.gcf().set_size_inches(12,12);


In [60]:
# check
data[data['rooms'] == 3].query('total_area < 201 and last_price < 25_000_000').plot(kind='scatter',
        y='last_price' , x='total_area', alpha=0.5, subplots=True, figsize=(15,8), c = 'b', s = 4)
plt.title('Диаграмма рассеяния — Общая площадь — цена трешки')


data[data['rooms'] == 3].query('total_area < 201 and last_price < 25_000_000').plot(kind='scatter', 
        y='last_price' , x='living_area', alpha=0.5, figsize=(15,8), c = 'r', s = 4)
plt.title('Диаграмма рассеяния — Жилая площадь — цена трешки');

In [61]:
# check
df_check = pd.read_csv('/datasets/real_estate_data.csv', sep='\t') 
df_check.info()
df_isna = df_check.query('airports_nearest.isna() | airports_nearest < 60000')
df_isna.info()

In [62]:
# check
df_check = pd.read_csv('/datasets/real_estate_data.csv', sep='\t') 
df_check.info()
df_isna = df_check[(df_check ['airports_nearest'] < 60000) | (df_check ['airports_nearest'].isna()) ]

df_isna.info()